In [9]:
# Constructs a quantum circuit g with parameters θ, then differentiates the recursive algorithm given in Section 5.1 of https://arxiv.org/abs/1112.2184 to obtain the gradient of p_θ(x) wrt θ, where x is a measurement of g|0>. The differentiation takes polynomial time due to memoization.
# We then compare our results to the finite difference gradient

using Revise
using Yao, FLOYao
using LinearAlgebra

nq = 49 #Number of qubits
layers = 2 #Number of brick-wall layers in the circuit
g = chain(nq)
for _ in 1:layers
    for i in 1:nq-1
        push!(g, rot(kron(nq, i => X, i+1 => X), 0.)) #Nearest-neighbor XX rotation gates
    end
    for i in 1:nq-1
        push!(g, rot(kron(nq, i => X, i+1 => Y), 0.)) #Nearest-neighbor XY rotation gates
    end
    for i in 1:nq
        push!(g, put(nq, i => Rz(0.))) #Single qubit Z rotation gates
    end
end

#Set g to have random parameters
p = rand(nparameters(g)).*2π
dispatch!(g, p)
nparams = nparameters(g)
dim = 2*nq
println("number of parameters: ", nparams)

⊗ = kron

function covariance_matrix(reg::MajoranaReg)
    nq = nqubits(reg)
    G = I(nq) ⊗ [0 1; -1 0]
    return reg.state * G * reg.state'
end

function majoranaindices2kron(nq, i, j) #Returns (im/2)γ_iγ_j, assuming that i≠j
    p = []
    c = (i % 2 == j % 2) ? 1 : -1
    a = min(i, j)
    b = max(i, j)
    first = (a+1) ÷ 2 
    last = (b+1) ÷ 2 
    if first == last #This means i=j-1 and j is even
        c = 1
        push!(p, first => Z)
    else
        if a % 2 == 0
            push!(p, first => X)
            c *= 1
        else
            push!(p, first => Y)
            c *= -1
        end
        for k in first+1:last-1
            push!(p, k => Z)
            c *= -1
        end
        if b % 2 == 0
            push!(p, last => Y)
        else
            push!(p, last => X)
        end
    end
    if i > j
        c *= -1
    end
    return c*kron(nq, p...)
end

function majorana_commutator(nq, i, j) #Returns [γ_i,γ_j]=2γ_iγ_j, due to the anti-commutation of Majorana operators. It needs to be an 'Add' object so that the Yao.expect' function can take it in as input.
    return Add(majoranaindices2kron(nq, i, j)) 
end

function update_opt!(reg::MajoranaReg, theta, b, temp_m, temp_grad_m, probabilities, grad_probabilities) #Evolves all matrices and probabilities and gradients by nq steps, in-place and optimally
    t_tot = 0
    for i in 1:nq
        t = time()
        if i > 1
            ni = b[i-1]
            cur_prob = probabilities[i-1]
            cur_grad_prob = grad_probabilities[:, i-1]
            cur_prefactor = (-1)^ni / (2*cur_prob)
            cur_grad_prefactor = (-1)^ni / (2*cur_prob^2)
            for p in 2*(i-1)+1:dim
                for q in p+1:dim
                    temp_grad_m[:,p,q] .-= cur_grad_prefactor * ((-cur_grad_prob * temp_m[2*(i-1)-1,p] * temp_m[2*(i-1),q]) .+ (cur_prob * (temp_grad_m[:,2*(i-1)-1,p]*temp_m[2*(i-1),q] .+ temp_m[2*(i-1)-1,p] * temp_grad_m[:,2*(i-1),q])))
                    temp_grad_m[:,p,q] .+= cur_grad_prefactor * ((-cur_grad_prob * temp_m[2*(i-1)-1,q] * temp_m[2*(i-1),p]) .+ (cur_prob * (temp_grad_m[:,2*(i-1)-1,q]*temp_m[2*(i-1),p] .+ temp_m[2*(i-1)-1,q] * temp_grad_m[:,2*(i-1),p])))
                end
            end
            for p in 2*(i-1)+1:dim
                for q in p+1:dim
                    temp_m[p,q] -= cur_prefactor * (temp_m[2*(i-1)-1,p] * temp_m[2*(i-1),q])
                    temp_m[p,q] += cur_prefactor * (temp_m[2*(i-1)-1,q] * temp_m[2*(i-1),p])
                end
            end
            ni = b[i]
            probabilities[i] = (1+(-1)^ni * temp_m[2*i-1, 2*i]) / 2
            grad_probabilities[:, i] = (-1)^ni * temp_grad_m[:,2*i-1, 2*i] / 2
        else
            dispatch!(g, theta)
            temp_m = covariance_matrix(apply(reg, g))
            ni = b[i]
            probabilities[i] = (1+(-1)^ni * temp_m[2*i-1, 2*i]) / 2
            for p in 1:dim
                for q in p+1:dim
                    ham = majorana_commutator(nq, p, q) #tr([γ_i,γ_j])
                    #profiler.jl
                    temp_grad_m[:,p,q] = expect'(ham, reg => g)[2]
                end
            end
            grad_probabilities[:, i] = (-1)^ni * temp_grad_m[:,2*i-1, 2*i] / 2
        end
        diff = (time() - t)
        t_tot += diff
        println("iteration $i: $diff")
    end
    println("total time: $t_tot")
end

function log_grad_opt(reg::MajoranaReg, theta, b, temp_m, temp_grad_m, probabilities, grad_probabilities) #Returns ∇_θlog(p_θ(b)), evaluated at 'theta' (parameters of circuit) and 'b' (measurement result); 'reg' is the initial register and must be of type MajoranaReg (e.g. FLOYao.zero_state(nq)). This uses the optimal updating function which is more efficient but still outputs the same thing as the original update! function.
    update_opt!(reg, theta, b, temp_m, temp_grad_m, probabilities, grad_probabilities)
    s = zeros(length(theta))
    for i in 1:nq
        s += grad_probabilities[:, i] / probabilities[i]
    end
    optimized_prob = probabilities
    return optimized_prob, s
end

reg = apply(FLOYao.zero_state(nq), g)
bitstr = measure(reg, nshots = 1)[1] #Random measurement of g|0>
println("measured outcome: $bitstr")
println("probability of measuring the above outcome: ", FLOYao.bitstring_probability(reg, bitstr)) #Uses FLOYao.bitstring_probability(reg, bitstr) which is known to be correct. We check this number against our algorithm output, to verify correctness.

T = Float64 #Can also be BigFloat, may experiment with other data types later
println("data type used in calculations: $T") 
println("note: the time (μs) taken for 'iteration i' refers to the time required for the algorithm to compute p_θ(x_i|x_1,...x_{i-1}) and ∇_θ(p_θ(x_i|x_1,...x_{i-1}))")

#Initializing temporary matrices and vectors.
temp_m = Matrix{T}(undef, dim, dim)
temp_grad_m = Array{T}(undef, nparams, dim, dim)
probabilities = Vector{T}(undef, nq)
grad_probabilities = Matrix{T}(undef, nparams, nq)

optimized_prob, optimized = log_grad_opt(FLOYao.zero_state(nq), p, bitstr, temp_m, temp_grad_m, probabilities, grad_probabilities)
println("The ith entry in the following vector is p_θ(x_i|x_1,...x_{i-1})")
println(optimized_prob)
println("the product of all entries in the above vector, should match the earlier probability computed using FLOYao.bitstring_probability: ", prod(optimized_prob))
println("The following vector is ∇_θ(log(p_θ(x))), evaluated at x = measured outcome")
optimized

number of parameters: 290
measured outcome: 0010001010000010001001001000001100110110110000111 ₍₂₎
probability of measuring the above outcome: 6.572217722828314943509355796447549839396566562277314055157507520556982144278418e-10
data type used in calculations: Float64
note: the time (μs) taken for 'iteration i' refers to the time required for the algorithm to compute p_θ(x_i|x_1,...x_{i-1}) and ∇_θ(p_θ(x_i|x_1,...x_{i-1}))
iteration 1: 9.44242000579834
iteration 2: 1.4111599922180176
iteration 3: 1.374161958694458
iteration 4: 1.5393040180206299
iteration 5: 1.4617111682891846
iteration 6: 1.452582836151123
iteration 7: 1.1886680126190186
iteration 8: 1.1635971069335938
iteration 9: 1.1632640361785889
iteration 10: 1.0557620525360107
iteration 11: 0.9877381324768066
iteration 12: 1.324187994003296
iteration 13: 0.8917109966278076
iteration 14: 0.8477609157562256
iteration 15: 0.7764089107513428
iteration 16: 0.7584681510925293
iteration 17: 0.7010140419006348
iteration 18: 0.750673055648

290-element Vector{Float64}:
  0.5712275802015896
  0.09639673903807366
 -0.7544990273941891
 -0.26467840941777154
 -0.4021704448552481
  0.18484022228806338
  0.14777820703716643
  0.15323758453714992
 -0.3259490394074839
 -0.5371720830238876
  ⋮
  2.31856731120661e-17
  6.207983388648282e-17
 -6.153376622282322e-17
 -3.644611150229159e-17
  4.026172415987515e-18
 -4.3329287638579323e-17
  1.6455398942425753e-17
 -4.733780400365111e-18
  1.503384524496456e-18

In [2]:
using BenchmarkTools

temp_m = Matrix{T}(undef, dim, dim)
temp_grad_m = Array{T}(undef, nparams, dim, dim)
probabilities = Vector{T}(undef, nq)
grad_probabilities = Matrix{T}(undef, nparams, nq)

dispatch!(g, p)
temp_m = covariance_matrix(apply(reg, g))
i = 1
b = bitstr
ni = b[i]
probabilities[i] = (1+(-1)^ni * temp_m[2*i-1, 2*i]) / 2
for p in 1:dim
    for q in p+1:dim
        ham = majorana_commutator(nq, p, q)
        temp_grad_m[:,p,q] = expect'(ham, reg => g)[2]
    end
end
grad_probabilities[:, i] = (-1)^ni * temp_grad_m[:,2*i-1, 2*i] / 2

function helper!(temp_m, temp_grad_m, probabilities, grad_probabilities)
    for i in 2:nq
        ni = b[i-1]
        cur_prob = probabilities[i-1]
        cur_grad_prob = grad_probabilities[:, i-1]
        cur_prefactor = (-1)^ni / (2*cur_prob)
        cur_grad_prefactor = (-1)^ni / (2*cur_prob^2)
        for p in 2*(i-1)+1:dim
            for q in p+1:dim
                temp_grad_m[:,p,q] .-= cur_grad_prefactor * ((-cur_grad_prob * temp_m[2*(i-1)-1,p] * temp_m[2*(i-1),q]) .+ (cur_prob * (temp_grad_m[:,2*(i-1)-1,p]*temp_m[2*(i-1),q] .+ temp_m[2*(i-1)-1,p] * temp_grad_m[:,2*(i-1),q])))
                temp_grad_m[:,p,q] .+= cur_grad_prefactor * ((-cur_grad_prob * temp_m[2*(i-1)-1,q] * temp_m[2*(i-1),p]) .+ (cur_prob * (temp_grad_m[:,2*(i-1)-1,q]*temp_m[2*(i-1),p] .+ temp_m[2*(i-1)-1,q] * temp_grad_m[:,2*(i-1),p])))
            end
        end
        for p in 2*(i-1)+1:dim
            for q in p+1:dim
                temp_m[p,q] -= cur_prefactor * (temp_m[2*(i-1)-1,p] * temp_m[2*(i-1),q])
                temp_m[p,q] += cur_prefactor * (temp_m[2*(i-1)-1,q] * temp_m[2*(i-1),p])
            end
        end
        ni = b[i]
        probabilities[i] = (1+(-1)^ni * temp_m[2*i-1, 2*i]) / 2
        grad_probabilities[:, i] = (-1)^ni * temp_grad_m[:,2*i-1, 2*i] / 2
    end
end

# @benchmark helper!(temp_m, temp_grad_m, probabilities, grad_probabilities)

helper! (generic function with 1 method)